In [2]:
import sys
sys.path.append("../src/")

import datetime
from tqdm.notebook import tqdm
import numpy as np

from tdms_io import get_reader_array, get_data_from_array, get_dir_properties

In [9]:
reader_array, timestamps = get_reader_array("/media/harry/Elements/DAS_Data/Myanmar_Quake/")
props = get_dir_properties("/media/harry/Elements/DAS_Data/Myanmar_Quake/")
prepro_para = {
        'cha1': 900,
        'cha2': 1300,
        'sps': props.get('SamplingFrequency[Hz]'),
        'spatial_res': props.get('SpatialResolution[m]'),
        'spatial_ratio': 1,
        'n_minute': 1,
        'freqmax': 49.9,
        'freqmin': 1,
}

t0 = datetime.datetime.strptime("2025-03-28T06:30:00", "%Y-%m-%dT%H:%M:%S")

180 files available from 2025-03-28 06:30:25.948612 to 2025-03-28 07:59:55.948612


In [ ]:
data = get_data_from_array(reader_array, prepro_para, t0, timestamps, duration=datetime.timedelta(minute=1))
print(data)

TypeError: function missing required argument 'year' (pos 1)